# Farm Field Instance Segmentation using MMDetection

**Goal:** To develop an instance segmentation model capable of identifying and delineating individual farm fields from aerial/satellite imagery. The secondary goal is to enable classification based on field size using post-processing techniques after successful segmentation.

**Methodology:** Utilize the MMDetection framework with a pre-trained model (e.g., Mask R-CNN) fine-tuned on a custom dataset of farm fields.

**Environment:** Kaggle Notebook with GPU acceleration.

## 2. Data Acquisition & Understanding

This section handles acquiring the dataset from Roboflow and performing an initial check. We will use the COCO format, as it's well-suited for MMDetection.

In [1]:
# Install Roboflow client
!pip install -q roboflow

# Import and authenticate
from roboflow import Roboflow
import os
import json

# --- IMPORTANT: Replace with your actual Roboflow API Key ---
# It's highly recommended to store API keys securely, e.g., using Kaggle Secrets
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("HE9CEH5JxJ3U0vXrQTOy") 
    print("Using Roboflow API Key from Kaggle Secrets.")
except:
    api_key = "YOUR_ROBOFLOW_API_KEY" # Fallback: Replace this placeholder directly
    if api_key == "YOUR_ROBOFLOW_API_KEY":
         print("WARNING: Using placeholder API Key. Please replace 'YOUR_ROBOFLOW_API_KEY' or set up Kaggle Secrets.")
    else:
         print("Using placeholder API Key from code variable.")
# -----------------------------------------------------------

rf = Roboflow(api_key=api_key)

# Download dataset in COCO format suitable for MMDetection
project = rf.workspace("sid-mp92l").project("final-detectron-2")
version = project.version(1)
dataset = version.download("coco") # Use standard COCO export

# Store the location of the downloaded dataset
# Roboflow downloads to a folder based on project name and version
# Check the output of the download cell to confirm the exact path
# It will likely be something like './final-detectron-2-1/'
# Let's determine it dynamically
dataset_location = dataset.location
print(f"Dataset downloaded to: {dataset_location}")

# Define paths to annotation files and image directories
# COCO format typically has train/valid/test splits
train_annot_path = os.path.join(dataset_location, "train", "_annotations.coco.json")
train_img_dir = os.path.join(dataset_location, "train")

valid_annot_path = os.path.join(dataset_location, "valid", "_annotations.coco.json")
valid_img_dir = os.path.join(dataset_location, "valid")

# (Optional) Test set paths if they exist
test_annot_path = os.path.join(dataset_location, "test", "_annotations.coco.json")
test_img_dir = os.path.join(dataset_location, "test")

# Basic Data Understanding: Check if files exist and load annotations
print("\nChecking downloaded files:")
print(f"Train annotations exist: {os.path.exists(train_annot_path)}")
print(f"Train images folder exists: {os.path.exists(train_img_dir)}")
print(f"Validation annotations exist: {os.path.exists(valid_annot_path)}")
print(f"Validation images folder exists: {os.path.exists(valid_img_dir)}")

# Load train annotations to inspect structure (optional)
if os.path.exists(train_annot_path):
    with open(train_annot_path, 'r') as f:
        train_data = json.load(f)
    print(f"\nNumber of training images: {len(train_data.get('images', []))}")
    print(f"Number of training annotations: {len(train_data.get('annotations', []))}")
    categories = train_data.get('categories', [])
    print(f"Categories: {categories}")
    if categories:
        num_classes = len(categories)
        print(f"Number of classes found: {num_classes}")
    else:
        num_classes = 0 # Handle case where categories might be missing
        print("Warning: Could not determine number of classes from annotations.")
else:
    print("\nCould not load training annotations for inspection.")
    num_classes = 1 # Assume 1 class ('farm') if annotations can't be read

# Assert that we have only one class ('farm') based on our prior discussion
# MMDetection needs num_classes = number of foreground classes
if categories:
     actual_num_classes = len(categories)
     if actual_num_classes == 1:
         print(f"Confirmed: Dataset has {actual_num_classes} class ('{categories[0]['name']}').")
         num_classes = 1 # Set for MMDetection config
     else:
         print(f"WARNING: Expected 1 class but found {actual_num_classes}. Check Roboflow export/annotations.")
         # Decide how to proceed - maybe default to 1 or raise an error
         num_classes = actual_num_classes # Or keep as 1 if you're sure

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=undefined&ref=undefined
loading Roboflow workspace...


RoboflowError: {
    "error": {
        "message": "This API key does not exist (or has been revoked).",
        "status": 401,
        "type": "OAuthException",
        "hint": "You may retrieve your API key via the Roboflow Dashboard. Go to Account > Roboflow Keys to retrieve yours.",
        "key": "YOUR_ROBOFLOW_API_KEY"
    }
}

## 3. Modeling - Environment Setup

Install MMDetection and its dependencies (`mmcv`, `mmengine`). We use `mim` (OpenMMLab's tool) for smoother installation, especially for `mmcv` which often needs compilation matching the PyTorch/CUDA version.

In [ ]:
# Install mim
!pip install -q openmim

# Use mim to install mmengine and mmcv
# This command automatically selects the correct mmcv binary for Kaggle's environment
!mim install mmengine mmcv

# Clone the MMDetection repository
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

# Install MMDetection from source
!pip install -e .

# Verify installation (optional)
import mmcv
import mmdet
import mmengine
print(f"MMCV version: {mmcv.__version__}")
print(f"MMDetection version: {mmdet.__version__}")
print(f"MMEngine version: {mmengine.__version__}")

# Go back to the working directory root for clarity
%cd ..

## 4. Modeling - Configuration

MMDetection uses Python configuration files (`.py`). We'll select a pre-trained instance segmentation model config (e.g., Mask R-CNN with ResNet-50 backbone) and modify it for our custom dataset.

**Key Modifications:**
1.  Update dataset paths (train/val annotations and image directories).
2.  Set the number of classes (`num_classes`) in the model's classification head.
3.  Specify the path to the pre-trained model weights (`load_from`).
4.  Adjust training parameters (e.g., epochs, learning rate) if needed.

In [ ]:
import os
from mmengine import Config

# --- Choose a base config ---
# Example: Mask R-CNN with ResNet-50 backbone, FPN neck, trained 1x schedule on COCO
# You can browse other configs in './mmdetection/configs/'
base_config_name = 'mask-rcnn_r50_fpn_1x_coco.py'
base_config_path = os.path.join('mmdetection', 'configs', 'mask_rcnn', base_config_name)

# --- Define path for our custom config ---
custom_config_name = 'mask-rcnn_r50_fpn_1x_farm.py'
custom_config_path = os.path.join('.', custom_config_name) # Save in working dir

# --- Load the base configuration ---
cfg = Config.fromfile(base_config_path)

# --- Modify the configuration ---

# 1. Dataset Paths & Type
cfg.data_root = dataset_location # Root directory of the dataset
cfg.dataset_type = 'CocoDataset' # Explicitly set dataset type

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.ann_file = os.path.join('train', '_annotations.coco.json') # Relative to data_root
cfg.train_dataloader.dataset.data_prefix = dict(img=os.path.join('train', '')) # Relative to data_root

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.ann_file = os.path.join('valid', '_annotations.coco.json') # Relative to data_root
cfg.val_dataloader.dataset.data_prefix = dict(img=os.path.join('valid', '')) # Relative to data_root

# Modify validation evaluator if necessary (usually fine)
cfg.val_evaluator.ann_file = os.path.join(cfg.data_root, 'valid', '_annotations.coco.json') # Needs full path or path relative to runtime

# If a test set exists and you want to configure it:
# cfg.test_dataloader.dataset.type = cfg.dataset_type
# cfg.test_dataloader.dataset.data_root = cfg.data_root
# cfg.test_dataloader.dataset.ann_file = os.path.join('test', '_annotations.coco.json')
# cfg.test_dataloader.dataset.data_prefix = dict(img=os.path.join('test', ''))
# cfg.test_evaluator.ann_file = os.path.join(cfg.data_root, 'test', '_annotations.coco.json')

# 2. Number of Classes
# Important: Needs to be changed in the model's head definition
# Find the roi_head section and update num_classes for bbox_head and mask_head
# The number of classes should be the number of foreground classes (1 for 'farm')
cfg.model.roi_head.bbox_head.num_classes = num_classes
cfg.model.roi_head.mask_head.num_classes = num_classes

# 3. Pre-trained Weights
# MMDetection automatically downloads weights if not found locally,
# using the URL specified in the base config's `load_from`.
# You can explicitly define it or rely on the default from the base config.
# Example of finding the default URL:
print(f"Attempting to load pre-trained weights from: {cfg.load_from}")
# No change needed here unless you want to use a different checkpoint.

# 4. Training Parameters (Optional Adjustments)
# Example: Reduce epochs for faster initial run
# cfg.train_cfg.max_epochs = 12 # Default is often 12 ('1x' schedule)
cfg.train_cfg.val_interval = 1 # Validate every epoch

# Set evaluation interval - useful for long training runs
# cfg.default_hooks.checkpoint.interval = 1 # Save checkpoint every epoch

# Modify learning rate (optional, defaults might be okay for fine-tuning)
# cfg.optim_wrapper.optimizer.lr = 0.02 / 8 # Example: Scale LR based on batch size if changed

# Set batch size (adjust based on GPU memory in Kaggle)
# Default might be 2 images per GPU. Check cfg.train_dataloader.batch_size
# cfg.train_dataloader.batch_size = 2
# cfg.train_dataloader.num_workers = 2 # Adjust based on Kaggle CPU resources

# Set a work directory for logs and checkpoints
cfg.work_dir = './work_dirs/farm_mask_rcnn'

# Create work_dir if it doesn't exist
os.makedirs(cfg.work_dir, exist_ok=True)

# --- Save the modified configuration ---
cfg.dump(custom_config_path)
print(f"Modified configuration saved to: {custom_config_path}")
print(f"\nKey changes applied:")
print(f"  - Dataset root: {cfg.data_root}")
print(f"  - Train annotations: {cfg.train_dataloader.dataset.ann_file}")
print(f"  - Val annotations: {cfg.val_dataloader.dataset.ann_file}")
print(f"  - Num classes (head): {cfg.model.roi_head.bbox_head.num_classes}")
print(f"  - Work directory: {cfg.work_dir}")


## 5. Modeling - Training

Use the MMDetection training script with our custom configuration file. Monitor the output for training loss and validation metrics (like mAP for segmentation).

In [ ]:
# Change directory to where the train script is located
%cd mmdetection

# Construct the training command
# --work-dir specifies where logs/models are saved, overriding the config file if needed
# We already set it in the config, so it's slightly redundant but ensures it's used.
train_command = f"python tools/train.py {os.path.join('..', custom_config_path)} --work-dir {os.path.join('..', cfg.work_dir)}"

print(f"Running training command:\n{train_command}")

# Execute the training command
!{train_command}

# Change back to the root working directory after training
%cd ..

## 6. Modeling - Evaluation & Prediction (Initial)

After training, you can evaluate the best performing checkpoint on the validation set or run inference on new images.

In [ ]:
# --- Evaluation (Optional) ---
# Usually done automatically during training if configured.
# You can re-run evaluation using test.py if needed:
# !python ./mmdetection/tools/test.py {custom_config_path} {path_to_best_checkpoint.pth} --show # For visualization

# --- Inference on a Sample Image ---
from mmdet.apis import init_detector, inference_detector
import mmcv
import matplotlib.pyplot as plt

# Find the latest checkpoint file in the work directory
latest_checkpoint = max([os.path.join(cfg.work_dir, f) for f in os.listdir(cfg.work_dir) if f.endswith('.pth')], key=os.path.getctime)
print(f"Using checkpoint: {latest_checkpoint}")

# Build the model from config and checkpoint
model = init_detector(custom_config_path, latest_checkpoint, device='cuda:0') # Use GPU

# Select a sample image for inference (e.g., the first validation image)
sample_image_path = os.path.join(valid_img_dir, os.listdir(valid_img_dir)[0]) # Get first image in valid dir
print(f"Running inference on: {sample_image_path}")

# Run inference
result = inference_detector(model, sample_image_path)

# Visualize the results (optional)
# The 'show_result_pyplot' function can draw bboxes and masks
# It might need the class names from your dataset
# Let's try to get class names if possible
class_names = []
if os.path.exists(valid_annot_path):
    with open(valid_annot_path, 'r') as f:
        valid_data = json.load(f)
    categories = valid_data.get('categories', [])
    class_names = [cat['name'] for cat in categories]
else: # Fallback if annotation file fails
     class_names = ['farm'] * num_classes # Assume based on previous logic

print(f"Using class names for visualization: {class_names}")

# Create a Matplotlib figure context
plt.figure(figsize=(15, 10))
# Visualize detection results
model.show_result(
    sample_image_path,
    result,
    score_thr=0.3, # Threshold to filter results
    show=True, # Show in the notebook output
    wait_time=0,
    # class_names=class_names # Pass class names if needed by the specific show_result implementation
    out_file=None # Don't save to file, display directly
)
# Due to interactive nature, sometimes direct plt.show() might be needed if image doesn't appear
# plt.show()


## 7. Next Steps (Deployment & Size Classification)

1.  **Refine Training:** Adjust hyperparameters (learning rate, epochs, augmentations) in the config file based on initial results to improve model performance (mAP).
2.  **Size Calculation:** Develop a post-processing script that takes the inference results (masks from `result` object), calculates the pixel area of each mask, and (if GSD is available) converts it to real-world area.
3.  **Size Classification:** Apply thresholds to the calculated areas to assign size categories (e.g., Small, Medium, Large Plot).
4.  **Deployment:** Package the trained model (`.pth` file) and the inference/post-processing script for deployment in the target environment (e.g., web service using Flask/FastAPI, integrated with satellite imagery API). Ensure the deployment environment handles fetching GSD information.